# diffusers stable-diffusion

* https://huggingface.co/stabilityai/stable-diffusion-2

* https://huggingface.co/docs/diffusers/main/en/training/dreambooth
* https://ngwaifoong92.medium.com/how-to-fine-tune-stable-diffusion-using-lora-85690292c6a8
* https://towardsdatascience.com/improving-diffusers-package-for-high-quality-image-generation-a50fff04bdd4

* https://stable-diffusion-art.com/controlnet

In [1]:
import os, glob
import random, math
import numpy as np
import pandas as pd
import torch
styles = pd.read_csv('stable_diffusion_artist_styles.csv')

In [2]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

#model_id = "runwayml/stable-diffusion-v1-5"
model_id = "stabilityai/stable-diffusion-2-1"
#model_id = "CompViz/stable-diffusion-2"

# Use the Euler scheduler here instead
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, safety_checker=None)
pipe = pipe.to("cuda")

/storage/local/stablediff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/storage/local/stablediff/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:


In [3]:
def prompt(prompt, n=1, style=None, path='.', negative_prompt=None):
    if style != None:
        prompt += ' by %s'%style
    if negative_prompt == None:
        negative_prompt = 'disfigured, lowres, bad anatomy, worst quality, low quality'
    for c in range(n):
        print (prompt)
        image = pipe(prompt, negative_prompt=negative_prompt).images[0]
        if not os.path.exists(path):
            os.makedirs(path)
        i=1
        imgfile = os.path.join(path,prompt[:90]+'_%s.png' %i)
        while os.path.exists(imgfile):
            i+=1
            imgfile = os.path.join(path,prompt[:90]+'_%s.png' %i)
        image.save(imgfile,'png')        
    return image

In [4]:
def get_words():
    import requests
    word_site = "https://www.mit.edu/~ecprice/wordlist.10000"
    response = requests.get(word_site)
    w = response.content.splitlines()
    w = [i.decode() for i in w]
    w = [i for i in w if len(i)>3]
    return w

randwords = get_words()

In [5]:
def random_image(artist,n=1,k=3,other='',path=None):
    for i in range(n):
        words = random.choices(randwords,k=k)
        print (words)
        txt = ' '.join(words)
        if other != '':
            txt+=', '+other
        if path == None:
            path = artist
        prompt(txt, style=artist, path=path)

In [6]:
name='inca'
media = pd.read_csv('stable_diffusion_media.csv').Name
#for m in media:
    #prompt(name,style=m,n=1,path=name)

In [ ]:
media

## look at varying styles

In [ ]:
random_image('Hedley & Klee, oil painting',n=20,path='test')

In [ ]:
names = styles.sample(20).Name
p='julian assange'
#for artist in names:
#    prompt(p,style='Alfred Sisley & '+artist,n=3,path='julian assange')
prompt(p,style='hedley, picasso & sisley',path='julian assange',n=10)

In [ ]:
#prompt('ireland',style='tolkien, robert bateman & erin hanson',path='test',n=10)
#prompt('dublin',style='alphonse mucha & cezanne',path='test',n=10)
prompt('lighthouse storm',style='Carl Rungius, Cezanne & Paul Hedley, painting',path='test',n=10)

In [ ]:
names = styles.sample(20).Name
p='happy child'
for name in names:
    prompt(p,style=name,path='test')

In [ ]:
#combinations
def combine_artists(txt, path, n=1):
    names = ['Paul Hedley','Paul Cezanne','Paul Klee','Robert Bateman','Giger',
            'Ray Harris-Ching','Erin Hanson','Carl Rungius','Inessa Garmash','Alphonse Mucha']
    for i in range(n):
        n1,n2,n3 = random.sample(names,k=3)
        prompt(txt,style='%s, %s & %s' %(n1,n2,n3),n=1,path=path)

combine_artists('magic boy', 'test', n=10)

In [ ]:
animals = [    
    "Penguin","Giraffe","Cat","Fox",
    "Lion","Elephant","Orangutan","Chimpanzee","Pig",
    "Bear","Hippopotamus","Gorilla","Zebra",
    "Ostrich","Badger","Gazelle","Raccoon"]

for a in animals:    
    combine_artists(a, 'animals', n=2)

## random combination of a dictionary of words

In [ ]:
#artist='paul hedley'
artist='robert bateman'
#artist='albert benois'
#artist='antoine blanchard'
#artist='phillipe druillet'
#artist='moebius'
#artist='vladimir volegov'
#artist='albrecht anker'
#artist='inessa garmash'
#artist='jrr tolkien'

themes = {'tolkien':(['middle earth'],['town','mountain','forest','tower','ruins','woodland','valley'],
                     ['horses','men','dragon','lady','warrior','elves',]),
        'ireland':(['ireland'],['monastic','castle','abbey','celtic','shamrock','green','ruins','cross'],['tree','oak','birch','moon','mountain','cliffs']),
        'nature':(['pond','forest','wood','lake','coast','meadow'],['cloudy','mountain','valley','grotto'],['evening','dusk','summer','spring','autumn','winter']),
        'woman':(['woman','lady'],['african','asian','russian','italian'],['beautiful','classical','elegant']),
        'scifi':(['steampunk','futuristic'],['machine','city','vehicle','car','motorbike','train'],['retro'])}
t='woman'
for i in range(10):
    words = themes[t]
    w = [random.choice(i) for i in words]
    w += ['impressionist']
    txt = '%s_'%t +' '.join(w)
    prompt(txt, style=artist, path=artist)

## film scenes

In [ ]:
artist='paul hedley'
modf = pd.read_csv('movies.csv')
for p in modf.sample(20).name:
    prompt(p+' impressionist', path='movie scenes',style=artist)

## fine tuned models

In [18]:
pipe = StableDiffusionPipeline.from_pretrained('/local/stablediff/', torch_dtype=torch.float16, safety_checker=None).to('cuda')
pipe.safety_checker = lambda images, clip_input: (images, False)

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
p=' photorealistic, high detail, high quality'
neg='cartoon, 3d, ((disfigured)), ((bad art)), ((deformed)),((extra limbs)),((close up)), weird colors, blurry, (((duplicate))), ((mutilated)), [out of frame], extra fingers, mutated hands, ((poorly drawn hands)), ((poorly drawn face)), (((mutation))), (((deformed))), ((ugly)), blurry, ((bad anatomy)), (((bad proportions))), ((extra limbs)), cloned face, (((disfigured))), out of frame, ugly, extra limbs, (bad anatomy), gross proportions, (malformed limbs), ((missing arms)), ((missing legs)), (((extra arms))), (((extra legs))), mutated hands, (fused fingers), (too many fingers), (((long neck))), Photoshop, video game, ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, mutation, mutated, extra limbs, extra legs, extra arms, disfigured, deformed, cross-eye, body out of frame, blurry, bad art, bad anatomy, 3d render'
for i in range(10):
    prompt(p,path='test',negative_prompt=neg)#,style='paul hedley')

In [31]:
def tile_images(image_paths, outfile, grid=False, tile_width=300):
    """Make tiled image"""

    from PIL import Image, ImageDraw
    images = [Image.open(path) for path in image_paths]
      
    ratio = images[0].height / images[0].width
    tile_height = int( tile_width * ratio )
    num_rows = int(math.sqrt(len(image_paths)))
    # Calculate number of cols
    num_columns = (len(images) + num_rows - 1) // num_rows

    tiled_width = num_columns * tile_width
    tiled_height = num_rows * tile_height
    tiled_image = Image.new("RGB", (tiled_width, tiled_height))

    for idx, image in enumerate(images):      
        row = idx // num_columns
        col = idx % num_columns
        x_offset = col * tile_width
        y_offset = row * tile_height
        tiled_image.paste(image.resize((tile_width, tile_height)), (x_offset, y_offset))
    if grid == True:
        draw = ImageDraw.Draw(tiled_image)
        # Draw borders around each tile
        for row in range(num_rows):
            for col in range(num_columns):
                x1 = col * tile_width
                y1 = row * tile_height
                x2 = x1 + tile_width
                y2 = y1 + tile_height
                draw.rectangle([x1, y1, x2, y2], outline=(0, 0, 0), width=3)  

    tiled_image.save(outfile)
    return tiled_image

In [ ]:
files = glob.glob('../../twitter/julian_assange/*.png')
files = glob.glob('elvis/*.png')
x = random.sample(files, 16)
tile_images(x, 'tiled.png', grid=True)